# Composition over Inheritance

为什么我们要用继承？

##  Cases and Problems

### 代码共享

1. 子类爆炸

如果想定制一个类的多于一个行为方面，通过继承共享代码是行不通的。它会导致子类爆炸。

2. 类和实例命名空间的混淆

子类的里的 `self.x` 到底来自哪里。**多层继承**，**多继承** 都会显著恶化这一问题。

更糟糕的情况，无法控制基类里暴露给用户的属性与方法。

3. 从属关系不明

问题 2 的后果之一

### 接口，抽象类

> 抽象数据类型（ADT）主要是为了收紧接口协议。你可以说你想要一个具有某些属性、方法的对象，而不关心其他的。在许多语言中，它们被称为接口，听起来没有那么高大上，所以我从现在开始将使用这个术语。

这个是我们经常使用的，同时也是副作用最小的

比如 Navigator 里定义了 `Optimizer` 的职能是什么，接下来所有的 `Optimizer` 都需要实现这一方法

In [1]:
from abc import ABC, abstractmethod
from typing import *

class BindingPose():
    pass


class PoseOptimizer(ABC):
    """Abstract class for generic binding pose optimizer."""

    @abstractmethod
    def optimize(
        self,
        binding_poses: Union[BindingPose, List[BindingPose]],  # TODO: support BindingPoseList
    ) -> List[BindingPose]:  # TODO: support BindingPoseList
        """Runs pose optimizer for a (list of) binding pose(s).

        Args:
            binding_poses: Binding pose(s) to be optimized.

        Returns:
            A list of `BindingPose` objects containing generated poses.
        """

再拓展一下，除了使用 abstract class，还有以下方法

1. 名义子类型

名义子类型意思是你必须告诉类型系统，你的类是一个接口定义的子类型。ABC 通常通过继承来实现这一点，但你也可以使用 register() 方法。

In [2]:
# Register

class FooOptimizer():
    def optimize(self, binding_poses):
        pass

assert isinstance(FooOptimizer(), PoseOptimizer)

AssertionError: 

In [3]:
PoseOptimizer.register(FooOptimizer)

assert isinstance(FooOptimizer(), PoseOptimizer)

2. 结构子类型

结构子类型也就是鸭子类型：如果你的类满足了一个协议的约束，它就会自动被认为是它的一个子类型。因此，一个类可以从各种包中实现许多协议而无需它们的存在！

In [4]:
from abc import ABC, abstractmethod
from typing import Union, List, Protocol, runtime_checkable

class BindingPose():
    pass


@runtime_checkable
class PoseOptimizer(Protocol):
    """Abstract class for generic binding pose optimizer."""

    def optimize(
        self,
        binding_poses: Union[BindingPose, List[BindingPose]],  # TODO: support BindingPoseList
    ) -> List[BindingPose]:  # TODO: support BindingPoseList
        """Runs pose optimizer for a (list of) binding pose(s).

        Args:
            binding_poses: Binding pose(s) to be optimized.

        Returns:
            A list of `BindingPose` objects containing generated poses.
        """

class BarOptimizer:
    def optimize(self, binding_poses):
        pass

assert isinstance(BarOptimizer(), PoseOptimizer)

可以思考一下，在这类情况下，继承的 **必要性** 在哪里？

### 特化

> 如果我们说一个类 B 特化了基类 A，其实就是说类 B 是具有额外属性的 A。一只狗是一种动物，A350 是一架客机。它们拥有基类的所有属性，并增加了属性、方法，或者只是在一个层次结构中增加了一个位置。

> 尽管这种简单很诱人，但它经常被错误地使用。最臭名昭著的错误是认为正方形是长方形的特化，因为从几何学上讲，它是一个特例。然而，正方形并不是拥有额外行为属性的长方形。

> 你不能在所有可以使用长方形的地方使用正方形，除非代码知道它也接受一个正方形8。如果你不能把一个对象当作其基类的实例来交互，你就违反了里氏替换原则9，你就不能写多态的代码。

（自己想的）长方形和正方形应该是平级的关系，对长方形干的操作对正方形也是可以的，实际上并没有从一个抽象的层级上去特化一个新的。

> 如果你仔细观察，你会发现上一节的接口是特化的一个特例。你总是把一个通用的 API 协议特化为一些具体的东西。关键的区别在于，抽象的数据类型是......嗯......抽象的。

案例

例如，设想你想把电子邮箱账户表示为类。它们都共享一些数据，比如它们在数据库中的 ID 和邮箱地址，此外，根据账户的类型，它们（可以）有额外的属性。重要的是，这些增加的属性和方法几乎没有改变现有的属性和方法。例如，一个在服务器上存储电子邮件的邮箱需要哈希过的密码作为登录信息，而一个接收电子邮件并只将其转发到另一个邮箱地址的账户则不需要10。

In [5]:
"""1. 为每种情况专门创建一个类"""

from uuid import UUID

class Mailbox:
    id: UUID
    addr: str
    pwd: str

class Forwarder:
    id: UUID
    addr: str
    targets: List[str]

In [6]:
"""2. 只创建一个类，把字段变成可选的
问题是非法状态必须不能表示，最终导致大量的 if-elif-else"""

import enum

class AddrType(enum.Enum):
    MAILBOX = "mailbox"
    FORWARDER = "forwarder"

class EmailAddr:
    type: AddrType
    id: UUID
    addr: str

    # Only useful if type == AddrType.MAILBOX
    pwd: Union[str, None]
    # Only useful if type == AddrType.FORWARDER
    target: Union[List[str], None]

In [7]:
"""3. 组合"""

class EmailAddr:
    id: UUID
    addr: str

class Mailbox:
    email: EmailAddr
    pwd: str

class Forwarder:
    email: EmailAddr
    targets: List[str]

In [8]:
"""4. 创建一个基类，然后特化它
子类的职责得到了限定，边界更清晰"""

class EmailAddr:
    id: UUID
    addr: str

class Mailbox(EmailAddr):
    pwd: str

class Forwarder(EmailAddr):
    targets: List[str]


## 组合模式

组合模式（Composite Pattern），又叫部分整体模式，是用于把一组相似的对象当作一个单一的对象。组合模式依据树形结构来组合对象，用来表示部分以及整体层次。这种类型的设计模式属于结构型模式，它创建了对象组的树形结构。

这种模式创建了一个包含自己对象组的类。该类提供了修改相同对象组的方式。

## 所以我们为什么需要引入组合

> 更多的时候，你需要的只是一个函数而已。

一个 case

我们的目标是实现仓库模式：一个允许你向数据仓库中添加和检索对象的类。此外，它还必须记住所有添加或检索的对象，并将其放在一个叫做 `seen` 的字段上

In [17]:
class Product:
    def __init__(self, sku) -> None:
        self.sku = sku


class AbstractRepository(ABC):
    seen: Set[Product]

    def __init__(self) -> None:
        self.seen = set()

    def add_product(self, product: Product) -> None:
        self._add_product(product)
        self.seen.add(product)

    def get_by_sku(self, sku: str) -> Optional[Product]:
        product = self._get_by_sku(sku)
        if product:
            self.seen.add(product)

        return product

    @abstractmethod
    def _add_product(self, product: Product):
        raise NotImplementedError

    @abstractmethod
    def _get_by_sku(self, sku: str) -> Optional[Product]:
        raise NotImplementedError

因此，每个子类都必须定义 _add_product() 和 _get_by_sku() 方法。然后用户调用 AbstractRepository 的add_product() 和 get_by_sku() 方法，这些方法又委托给子类的 _add_product() 和 _get_by_sku() ，同时记住它看到过哪些 Product 类型的对象。

**它将接口的定义与子类的共享代码混在一起。**

**更实际的问题是，如果你想理解代码的工作流，由于去向和来源在类层级中来回横跳，这会很困难。**

如何修改

In [18]:
class Repository(Protocol):
    def add_product(self, product: Product) -> None: ...
    def get_by_sku(self, sku: str) -> Optional[Product]: ...

In [19]:
class DictRepository:
    _storage: Dict[str, Product]

    def __init__(self):
        self._storage = {}

    def add_product(self, product: Product) -> None:
        self._storage[product.sku] = product

    def get_by_sku(self, sku: str) -> Optional[Product]:
        return self._storage.get(sku)

In [21]:
class TrackingRepository:
    _repo: Repository
    seen: Set[Product]

    def __init__(self, repo: Repository) -> None:
        self._repo = repo
        self.seen = set()

    def add_product(self, product: Product) -> None:
        self._repo.add_product(product)
        self.seen.add(product)

    def get_by_sku(self, sku: str) -> Optional[Product]:
        product = self._repo.get_by_sku(sku)
        if product:
            self.seen.add(product)

        return product

这个类由两个东西组合而成：一个只知道是实现了 Repository 的对象，和一些 Products。如果你在 `_repo` 属性上使用其他未被 Repository 接口承诺的东西，无需执行代码，类型检查器就会对你报警。

另一方面，我们最终得到了两个独立的小类，它们之间唯一的协定是一个严格的、明确的接口。这不仅易于阅读和理解，而且也易于测试。